***GENERATED CODE FOR regression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Course', 'transformation_label': 'String Indexer'}], 'feature': 'Course', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Business', 'max': 'Others', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Course'}, {'feature_label': 'Course', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Course')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sleep_Quality', 'transformation_label': 'String Indexer'}], 'feature': 'Sleep_Quality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Poor', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sleep_Quality'}, {'feature_label': 'Sleep_Quality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sleep_Quality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Physical_Activity', 'transformation_label': 'String Indexer'}], 'feature': 'Physical_Activity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Moderate', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Physical_Activity'}, {'feature_label': 'Physical_Activity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Physical_Activity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diet_Quality', 'transformation_label': 'String Indexer'}], 'feature': 'Diet_Quality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'Poor', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Diet_Quality'}, {'feature_label': 'Diet_Quality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Diet_Quality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Social_Support', 'transformation_label': 'String Indexer'}], 'feature': 'Social_Support', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Moderate', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Social_Support'}, {'feature_label': 'Social_Support', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Social_Support')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Relationship_Status', 'transformation_label': 'String Indexer'}], 'feature': 'Relationship_Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'In a Relationship', 'max': 'Single', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Relationship_Status'}, {'feature_label': 'Relationship_Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Relationship_Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Substance_Use', 'transformation_label': 'String Indexer'}], 'feature': 'Substance_Use', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '494', 'mean': '', 'stddev': '', 'min': 'Frequently', 'max': 'Occasionally', 'missing': '6', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Substance_Use'}, {'feature_label': 'Substance_Use', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Substance_Use')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Counseling_Service_Use', 'transformation_label': 'String Indexer'}], 'feature': 'Counseling_Service_Use', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Frequently', 'max': 'Occasionally', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Counseling_Service_Use'}, {'feature_label': 'Counseling_Service_Use', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Counseling_Service_Use')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Family_History', 'transformation_label': 'String Indexer'}], 'feature': 'Family_History', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Family_History'}, {'feature_label': 'Family_History', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Family_History')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Chronic_Illness', 'transformation_label': 'String Indexer'}], 'feature': 'Chronic_Illness', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Chronic_Illness'}, {'feature_label': 'Chronic_Illness', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Chronic_Illness')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Extracurricular_Involvement', 'transformation_label': 'String Indexer'}], 'feature': 'Extracurricular_Involvement', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Moderate', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Extracurricular_Involveme...'}, {'feature_label': 'Extracurricular_Involvement', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Extracurricular_Involvement')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Residence_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Residence_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Off-Campus', 'max': 'With Family', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Residence_Type'}, {'feature_label': 'Residence_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Residence_Type')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run regressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	studentsmentalhealthsurvey = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/students_mental_health_survey.csv', 'filename': 'students_mental_health_survey.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.61.134', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(studentsmentalhealthsurvey)

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionautofe = TransformationMain.run(studentsmentalhealthsurvey,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "23.06", "stddev": "3.76", "min": "18", "max": "35", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Course", "transformation_label": "String Indexer"}], "feature": "Course", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Business", "max": "Others", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Course"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CGPA", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "497", "mean": "3.48", "stddev": "0.3", "min": "2.44", "max": "4.0", "missing": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CGPA"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Stress_Level", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.45", "stddev": "1.59", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "Stress_Level"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Depression_Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.24", "stddev": "1.64", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "Depression_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Anxiety_Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.25", "stddev": "1.58", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "Anxiety_Score"}, {"transformationsData": [{"feature_label": "Sleep_Quality", "transformation_label": "String Indexer"}], "feature": "Sleep_Quality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Poor", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sleep_Quality"}, {"transformationsData": [{"feature_label": "Physical_Activity", "transformation_label": "String Indexer"}], "feature": "Physical_Activity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Moderate", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Physical_Activity"}, {"transformationsData": [{"feature_label": "Diet_Quality", "transformation_label": "String Indexer"}], "feature": "Diet_Quality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "Poor", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Diet_Quality"}, {"transformationsData": [{"feature_label": "Social_Support", "transformation_label": "String Indexer"}], "feature": "Social_Support", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Moderate", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Social_Support"}, {"transformationsData": [{"feature_label": "Relationship_Status", "transformation_label": "String Indexer"}], "feature": "Relationship_Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "In a Relationship", "max": "Single", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Relationship_Status"}, {"transformationsData": [{"feature_label": "Substance_Use", "transformation_label": "String Indexer"}], "feature": "Substance_Use", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "494", "mean": "", "stddev": "", "min": "Frequently", "max": "Occasionally", "missing": "6", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Substance_Use"}, {"transformationsData": [{"feature_label": "Counseling_Service_Use", "transformation_label": "String Indexer"}], "feature": "Counseling_Service_Use", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Frequently", "max": "Occasionally", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Counseling_Service_Use"}, {"transformationsData": [{"feature_label": "Family_History", "transformation_label": "String Indexer"}], "feature": "Family_History", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Family_History"}, {"transformationsData": [{"feature_label": "Chronic_Illness", "transformation_label": "String Indexer"}], "feature": "Chronic_Illness", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Chronic_Illness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Financial_Stress", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.39", "stddev": "1.69", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "Financial_Stress"}, {"transformationsData": [{"feature_label": "Extracurricular_Involvement", "transformation_label": "String Indexer"}], "feature": "Extracurricular_Involvement", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Moderate", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Extracurricular_Involveme..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Semester_Credit_Load", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "22.48", "stddev": "4.46", "min": "15", "max": "29", "missing": "0"}, "updatedLabel": "Semester_Credit_Load"}, {"transformationsData": [{"feature_label": "Residence_Type", "transformation_label": "String Indexer"}], "feature": "Residence_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Off-Campus", "max": "With Family", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Residence_Type"}]}))

	#transformationPostExecutionHook(regressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionautofe, ["Age", "CGPA", "Stress_Level", "Depression_Score", "Financial_Stress", "Semester_Credit_Load", "Course_stringindexer", "Gender_stringindexer", "Sleep_Quality_stringindexer", "Physical_Activity_stringindexer", "Diet_Quality_stringindexer", "Social_Support_stringindexer", "Relationship_Status_stringindexer", "Substance_Use_stringindexer", "Counseling_Service_Use_stringindexer", "Family_History_stringindexer", "Chronic_Illness_stringindexer", "Extracurricular_Involvement_stringindexer", "Residence_Type_stringindexer"], "Anxiety_Score")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

